In [ ]:
from langchain_openai import ChatOpenAI
import openai
from openai import OpenAI
import re
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.schema import Document
from langchain_community.llms import Ollama



In [ ]:
my_api_key = "input key"
llm_from_openAI = ChatOpenAI(api_key = my_api_key)
llm = Ollama(model="llama2")
embeddings = OpenAIEmbeddings(api_key = my_api_key)

In [ ]:
llm_from_openAI.invoke("你讀得懂中文嗎??")

In [ ]:
llm_from_openAI.invoke("how can langsmith help with testing?")

In [ ]:
llm.invoke("你讀得懂中文嗎?")

In [ ]:
llm.invoke("成功大學在哪?")

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

# Input template
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a course recommendation system of NCKU."),
    ("user", "{input}")
])

In [ ]:
from langchain_core.output_parsers import StrOutputParser

# 提供 output 轉換成 string type
output_parser = StrOutputParser()

In [ ]:
chain = prompt | llm | output_parser
chain_OpenAI = prompt | llm_from_openAI | output_parser

In [ ]:
import pandas as pd
from langchain_community.document_loaders import JSONLoader
import json
from pathlib import Path
from pprint import pprint


file_path='/Users/linchihhsien/Documents/Arduino/爬蟲/langchain-practice/courses_data.json'
#file_path='/Users/linchihhsien/Documents/Arduino/爬蟲/langchain-practice'
data = json.loads(Path(file_path).read_text())

In [ ]:
pprint(Path(file_path).read_text())

In [ ]:
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter

client = OpenAI(api_key=my_api_key)

In [ ]:
split_text = []
full_texts = []  # List to store all combined texts
for course in data:
    text_blocks = [
        course['Course Name'],
        course['Grading'],
        course['Department'],
        course['Teaching Strategies'],
        course['Syllabus Progress'],
        course['Course Description']
    ]

    split_text.append(text_blocks)
    full_text = "\n".join(text_blocks)
    full_texts.append(full_text)  # Add the combined text to the list

# Now full_texts contains all the combined text blocks
for text in full_texts:
    print(text)  # This will print all combined texts after the loop

In [ ]:
print(full_texts)

In [ ]:
text_splitter = RecursiveCharacterTextSplitter()

documents = text_splitter.create_documents(full_texts)
for i in range(5):
    print(documents[i])


In [ ]:
import pandas as pd

# 读取CSV文件
df = pd.read_csv('courseScore.csv')

# 显示数据以确认读取成功
print(df.head())

In [ ]:
vector = FAISS.from_documents(documents, embeddings)

In [ ]:

response = llm_from_openAI.invoke("你可以推薦一些成大管理學相關的課程嗎？")
print(response)

In [ ]:
chain.invoke("你可以推薦一些成大管理學相關的課程嗎？")#不能跑

In [ ]:
chain_OpenAI.invoke("你可以推薦一些成大管理學相關的課程嗎？")#不能跑

In [ ]:
from langchain.chains import create_retrieval_chain

retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, chain)
retrieval_chain_OpenAI = create_retrieval_chain(retriever, chain_OpenAI)

In [ ]:
retrieval_chain.invoke({"input": "你可以推薦一些成大管理學相關的課程嗎？"})

In [ ]:
answer = retrieval_chain_OpenAI.invoke({"input": "你可以推薦一些成大管理學相關的課程嗎？"})

In [ ]:
print(answer['answer'])
for context in answer['context']:
    print(context)

In [ ]:
answer2 = retrieval_chain_OpenAI.invoke({'input': "你可以跟我介紹工資係的管理學課的課程評分方式嗎？"})

In [ ]:
print(answer2['answer'])
for context in answer['context']:
    print(context)

In [ ]:
# 定義你的提示模板
prompt = """
課程背景：管理學院中所有系所所開設課程中，出席情況通常會在一定的比例下影響總分。
問題：工資管系統計軟體與繪圖這門課中出席分數佔比多少？
"""

# 使用提示模板來調用 AI 系統以獲取回應
answer = retrieval_chain_OpenAI.invoke({'input': prompt})

# 印出答案及任何額外的背景信息
print(answer['answer'])
for context in answer['context']:
    print(context)


In [ ]:
prompt = """The following are excerpts from conversations with an AI
assistant. The assistant is typically concise and Accurate, producing
accurate responses to the users questions. Here are some
examples: 

User: 工資管系統計軟體與繪圖這門課中出席分數佔比多少？
AI: 工資管系統計軟體與繪圖這門課中出席分數佔比為20%

User: 工資管系計算機該論這門課中出席分數佔比多少？
AI: 因為在我的資料庫中工資管系計算機該論這門課的課程大綱中並沒有描述出席分數的比重因此我無法回答你

User: 生物統計這門課中出席分數佔比多少？
課程背景：生物統計這門課中不包含出席分數
AI: 這門課程中出席分數並不會影響總分

User: 保險數學這門課中出席分數佔比多少？
AI: """

# 使用提示模板來調用 AI 系統以獲取回應
answer = retrieval_chain_OpenAI.invoke({'input': prompt})

# 印出答案及任何額外的背景信息
print(answer['answer'])
for context in answer['context']:
    print(context)
